In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
### <changed>
import os 
path_folder = os.getcwd()
path_folder = os.path.join(path_folder,"data","raw")
path_file_offer_acceptance_offers = os.path.join(path_folder, "offer_acceptance_offers.csv")
path_file_offer_acceptance_orders = os.path.join(path_folder, "offer_acceptance_orders.csv")

data = pd.read_csv(path_file_offer_acceptance_offers)
### </changed>

def conv_time(string):
    datetime_object = datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
    return datetime_object

data['CREATED_ON_HQ'] = data['CREATED_ON_HQ'].apply(conv_time)
data.head()

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID
0,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:54:13,9546.0,pool,True,False,False,False,False,False,False,True,True
1,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:27,9159.0,pool,True,True,False,True,False,True,True,True,True
2,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:28,9288.0,pool,False,False,False,False,False,False,False,False,True
3,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 08:57:40,9159.0,pool,False,True,False,True,False,False,False,False,True
4,16badfc6202cb3f8889e0f2779b19218af4cbb736e56ac...,"[\n ""ce3548db155049cc1ccce2da041cec607942e4f7...",2021-11-03 09:05:43,11610.0,pool,False,False,False,False,False,False,False,False,True


##### group offers by reference ID and exclude offers with more than one reference ID

In [3]:
### <changed>
# you're using the older versions of the csv's; Matt posted corrected ones where the transport mode is in the orders table,
# not the offers
FTL_data = data#[data['TRANSPORT_MODE'] == 'FTL'] # comment this out

# you forgot to include dropping he more than 1 reference id
FTL_data = FTL_data[((FTL_data["REFERENCE_NUMBER"].str.count(",")) < 1)] 
### </changed>

FTL_data = FTL_data.drop(['IS_OFFER_APPROVED', 'AUTOMATICALLY_APPROVED', 'MANUALLY_APPROVED'], axis=1)

FTL_data.shape

(594425, 11)

In [4]:
FTL_data.head()

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID
823,8722b305341fb1e3f4215fe1ad2322c634d4f21528c9e4...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 06:26:02,2838.0,pool,False,False,False,False,True,True
12904,78b4dce77c1c7927a138fcae25504a4397be895f1fb921...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-03 12:05:38,4515.0,pool,True,False,False,False,False,True
12905,dc7d6256d124bcc9244d6d0e437beabded4b25d4fbf77c...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-03 12:27:32,3870.0,pool,False,False,False,False,False,True
12906,58d99ed594dbcc48ef0eb08c97e05f972a29bd727644b0...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-03 12:25:33,3225.0,pool,True,True,False,False,False,True
112997,8722b305341fb1e3f4215fe1ad2322c634d4f21528c9e4...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 06:26:02,2838.0,pool,False,False,False,False,True,True


In [5]:
# Is this a typo? should it be orders, not offers? (Decided not to rechange the naming)
offers = pd.read_csv(path_file_offer_acceptance_orders)
offers['ORDER_DATETIME_PST'] = offers['ORDER_DATETIME_PST'].apply(conv_time)
offers['PICKUP_DEADLINE_PST'] = offers['PICKUP_DEADLINE_PST'].apply(conv_time)

offers['diff_ORDER_PICKUP_DEADLINE'] = (offers['PICKUP_DEADLINE_PST'] - offers['ORDER_DATETIME_PST'])
offers.head()

c:\users\benson\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,10,11,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,REFERENCE_NUMBER,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE
0,"[\n ""291ba58f4d48916df29008a6e451994650458661...",2022-09-07 10:07:05,2022-09-12 11:00:00,SCHEDULE,386,080,1035.0,22.0,True,True,False,True,True,0.0,True,3864.3240,FTL,5 days 00:52:55
1,"[\n ""e97bdf5ad99011a878f18c10b0df1f663f6b8f4a...",2022-05-19 09:21:39,2022-05-24 13:00:00,NONE,670,630,414.0,45.0,True,True,False,False,True,0.0,True,1411.1181,FTL,5 days 03:38:21
2,"[\n ""3cad7c1b5baf14968a6ee7c9b3649cb56193a40c...",2022-07-06 09:45:30,2022-07-08 14:00:00,SCHEDULE,952,853,703.0,20.0,True,True,False,False,False,0.0,True,2385.0681,FTL,2 days 04:14:30
3,"[\n ""c407879a0fc7dfe8dd40b8e0a8952eea0c5c850b...",2022-09-12 05:38:14,2022-09-19 10:30:00,SCHEDULE,381,346,753.0,52.0,True,False,False,False,False,0.0,False,2846.1141,FTL,7 days 04:51:46
4,"[\n ""e4ffed52c673af27ab35fcf4bb45e77becbbd95e...",2022-06-21 10:52:48,2022-07-05 13:00:00,SCHEDULE,917,953,345.0,48.0,True,True,False,False,False,0.0,False,1695.6405,FTL,14 days 02:07:12


In [6]:
### <changed>
# you're using the older versions of the csv's; Matt posted corrected ones where the transport mode is in the orders table,
offers = offers[offers["TRANSPORT_MODE"]=="FTL"]
### </changed>


joined = pd.merge(FTL_data, offers, on='REFERENCE_NUMBER', how='outer')
joined.sort_values(by = ['REFERENCE_NUMBER', 'CREATED_ON_HQ'], ascending = [True, True], na_position = 'first')
joined = joined[joined['OFFER_TYPE'] == 'quote']
joined.head(10)

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,...,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE
2,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:31:39,2064.0000,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:36:27,2064.0000,quote,False,False,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:28:33,2193.0000,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,d26ae7320c7e0be8e94962a8674e42a917c73cac667418...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 07:18:39,1328.7000,quote,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,730b498661c098017bbbcdcd229d1a2f707f5ab533fe55...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:37:34,2322.0000,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,8722b305341fb1e3f4215fe1ad2322c634d4f21528c9e4...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 06:23:02,2580.0000,quote,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
14,872119f809e53e0f8414eb391e24957d8b7f6f170c497d...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-09-30 03:31:15,2064.0000,quote,True,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
15,58d99ed594dbcc48ef0eb08c97e05f972a29bd727644b0...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-04 08:12:43,2133.9438,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
16,58d99ed594dbcc48ef0eb08c97e05f972a29bd727644b0...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-01 05:38:34,2451.0000,quote,True,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
17,58d99ed594dbcc48ef0eb08c97e05f972a29bd727644b0...,"[\n ""e07d64df5136bbeee8e09861abb49fb418191444...",2022-10-04 09:36:24,3225.0000,quote,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


#### total revenue from flock algorithm

In [7]:
true_joined = joined[joined['LOAD_DELIVERED_FROM_OFFER'] == True]
# all_sales = true_joined.groupby('REFERENCE_NUMBER').sum()
total_sales = true_joined['RATE_USD'].mean()
total_sales

2804.4188151292215

In [8]:
test_data = joined[:10000]
test_data

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,...,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,diff_ORDER_PICKUP_DEADLINE
2,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:31:39,2064.0,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:36:27,2064.0,quote,False,False,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,da26b77becf8bcc17aa4e59a0205d8910373e5a25e0d79...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:28:33,2193.0,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,d26ae7320c7e0be8e94962a8674e42a917c73cac667418...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 07:18:39,1328.7,quote,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,730b498661c098017bbbcdcd229d1a2f707f5ab533fe55...,"[\n ""ae7c9e922c6757aa7f182c8bd8b194d2ce0bb9ad...",2022-08-30 09:37:34,2322.0,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10003,24ba4243914d44f9f3a697989745b683c7a352bea930d3...,"[\n ""ed1938e57c5072eafb7b91df0a60adfde17d9163...",2022-04-25 07:08:00,5160.0,quote,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10004,929aa622b6a06994b83ead5da8db1ec18965dd051c4c82...,"[\n ""ed1938e57c5072eafb7b91df0a60adfde17d9163...",2022-04-21 07:37:47,4644.0,quote,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10005,abb26da6b9213bde37d0ac5721a9b4307f81509d10a833...,"[\n ""ed1938e57c5072eafb7b91df0a60adfde17d9163...",2022-04-25 07:57:30,5418.0,quote,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10006,46f4f325375a96211862a36f96dff19eca635dc2371b51...,"[\n ""5c402eb3eb2c11f184fa535f019349be24822cd9...",2022-07-21 14:42:49,4773.0,quote,False,False,True,True,False,...,True,False,False,True,True,0.0,False,4335.2256,FTL,0 days 08:58:05


In [9]:
ans = []
for n in test_data['REFERENCE_NUMBER'].unique():
    SecMeth = int(len(test_data[test_data['REFERENCE_NUMBER'] == n]) / np.e)
    GB = test_data[test_data['REFERENCE_NUMBER'] == n].sort_values(by='CREATED_ON_HQ', ascending=True)
    record = np.array(GB['RATE_USD'])[SecMeth]
    offer = 0
    for num in np.array((GB['RATE_USD'])[SecMeth:-1]):
        if num > record:
            offer = num
            break
        else:
            offer = np.array(GB['RATE_USD'])[-1]
        ans.append(offer)
            

In [10]:
np.mean(ans)

2888.053221170396

In [89]:
np.array(FTL_data['CREATED_ON_HQ'])

array(['2021-11-03 09:05:43', '2021-11-03 08:54:13',
       '2021-11-03 08:57:27', ..., '2021-11-15 11:25:58',
       '2021-04-26 10:00:23', '2022-09-02 05:33:02'], dtype=object)

In [ ]:
# turn over 37% of the offers and then after that choose the next offer that is a record

In [191]:
#1 on the sample size i did it made about 300$ more per shipment
#2 using the floor round is a bit more conservative and boosts the average rate by about 100$
#3 you have to know the number of offers before you calculate when to stop

#1 the secretary approach now is conditional because the second stopping point depends on when we stop for 
#the first pool shipment
#2 Somehow we have to include a manual point of when to stop for the first of the 2 pool shipments because if 
#we wait too long to accept our first pool offer then we have less to choose from for our second pool offer

In [ ]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(joined['B'])
# Drop column B as it is now encoded
df = df.drop('B',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df  

In [71]:
df = pd.merge(FTL_data, offers, on='REFERENCE_NUMBER', how='inner')
df.sort_values(by = ['REFERENCE_NUMBER', 'CREATED_ON_HQ'], ascending = [True, True], na_position = 'first')
df = df[df['OFFER_TYPE'] == 'quote']
df.head()

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,...,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE_y,diff_ORDER_PICKUP_DEADLINE
0,ba3be1d6cdc455d085fabeafa3b5f12ce439c09fb7f25f...,"[\n ""46d8930a62fddd24e3d682d7231b19a4e2f8b5d8...",2021-08-31 10:09:23,3483.0,quote,False,False,True,True,False,...,True,False,False,True,True,0.0,False,3528.2016,FTL,1 days 23:49:39
1,d302498475ec90c0b6da20bb1067dbcd62a40e59784727...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 05:41:19,4902.0,quote,False,False,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
2,cb7593a6adb4b6339f9b0d794acc8903819db9e7c57624...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 09:51:09,2064.0,quote,False,False,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
3,8cd6cb7e1601eb5dcd0a0d3d8f62c2dd8ee483819f9130...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 07:48:15,1935.0,quote,False,True,False,False,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12
4,10bb64fa1d284779a38ce9cc96b75ac7aa48c6a4bec7b2...,"[\n ""3b74ebf4c440a5ff141cafdc9b3e20230fc18360...",2021-10-08 09:56:44,1806.0,quote,False,False,True,True,False,...,True,False,False,True,True,0.0,False,1995.4623,FTL,12 days 07:10:12


In [83]:
df['diff_ORDER_PICKUP_DEADLINE'] = df['diff_ORDER_PICKUP_DEADLINE'].dt.total_seconds()

In [84]:
df.corr()

,RATE_USD,SELF_SERVE,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,LOAD_BAR_COUNT,ESTIMATED_COST_AT_ORDER,diff_ORDER_PICKUP_DEADLINE
RATE_USD,1.000000,0.001978,-0.001574,-0.003495,-0.003435,0.001596,-0.004666,0.003803,0.004150,-0.000490,0.005820,-0.000695
SELF_SERVE,0.001978,1.000000,-0.055376,-0.089712,-0.092081,0.616471,0.065340,-0.016211,-0.089418,0.015624,-0.036166,0.035946
WAS_EVER_UNCOVERED,-0.001574,-0.055376,1.000000,-0.114766,-0.112455,-0.053399,-0.021033,-0.022027,0.011705,0.001465,-0.012519,0.063504
COVERING_OFFER,-0.003495,-0.089712,-0.114766,1.000000,0.981879,-0.110304,0.126409,-0.055364,0.002011,-0.002315,-0.031002,-0.034946
LOAD_DELIVERED_FROM_OFFER,-0.003435,-0.092081,-0.112455,0.981879,1.000000,-0.110770,0.124410,-0.056506,0.001845,-0.002616,-0.032394,-0.043247
RECOMMENDED_LOAD,0.001596,0.616471,-0.053399,-0.110304,-0.110770,1.000000,0.040611,-0.007727,-0.069412,0.010627,-0.005629,0.025546
VALID,-0.004666,0.065340,-0.021033,0.126409,0.124410,0.040611,1.000000,-0.005376,0.001777,-0.002624,-0.001686,-0.004469
APPROXIMATE_DRIVING_ROUTE_MILEAGE,0.003803,-0.016211,-0.022027,-0.055364,-0.056506,-0.007727,-0.005376,1.000000,0.049537,0.038582,0.895278,-0.040559
PALLETIZED_LINEAR_FEET,0.004150,-0.089418,0.011705,0.002011,0.001845,-0.069412,0.001777,0.049537,1.000000,-0.001264,0.075916,0.033462
LOAD_BAR_COUNT,-0.000490,0.015624,0.001465,-0.002315,-0.002616,0.010627,-0.002624,0.038582,-0.001264,1.000000,0.037325,0.051645


In [57]:
df.columns

Index(['CARRIER_ID', 'REFERENCE_NUMBER', 'CREATED_ON_HQ', 'RATE_USD',
       'OFFER_TYPE', 'SELF_SERVE', 'WAS_EVER_UNCOVERED', 'COVERING_OFFER',
       'LOAD_DELIVERED_FROM_OFFER', 'RECOMMENDED_LOAD', 'TRANSPORT_MODE_x',
       'VALID', 'ORDER_DATETIME_PST', 'PICKUP_DEADLINE_PST',
       'DELIVERY_TIME_CONSTRAINT', 'ORIGIN_3DIGIT_ZIP',
       'DESTINATION_3DIGIT_ZIP', 'APPROXIMATE_DRIVING_ROUTE_MILEAGE',
       'PALLETIZED_LINEAR_FEET', 'FD_ENABLED', 'EXCLUSIVE_USE_REQUESTED',
       'HAZARDOUS', 'REEFER_ALLOWED', 'STRAIGHT_TRUCK_ALLOWED',
       'LOAD_BAR_COUNT', 'LOAD_TO_RIDE_REQUESTED', 'ESTIMATED_COST_AT_ORDER',
       'TRANSPORT_MODE_y'],
      dtype='object')

In [59]:
df['DESTINATION_3DIGIT_ZIP'].nunique()

822